## Testing speech identification from default mic
1. deploy speech service in azure.
2. run the following code (speak in normal pace and long sentence to let the languae identification to match speech to languages)

```
Speak into your microphone to detect the language...
Listening... Press Ctrl+C to stop.
Detected Language: zh-CN
Recognized Text: 大家好，欢迎到慕尼黑的体验中心.
Ctrl+C
```

You need to stop the previous language identification and rerun for new identification
```
Detected Language: en-US
Recognized Text: Good morning，everyone。welcome to the Munich experience center
Ctrl+C
```

You need to stop the previous language identification and rerun for new identification
```
Detected Language: de-DE
Recognized Text: Herzlich Willkommen zu Münchener Erlebniszentrum.
```

In [ ]:
from dotenv import dotenv_values
import os

current_path = os.getcwd()
env_file_path = os.path.join(current_path, "envs", "azurespeech.env")

# Load the environment variables from the .env file
config = {**dotenv_values(env_file_path)}

In [ ]:
import azure.cognitiveservices.speech as speechsdk

# Replace with your Azure Speech resource key and region
speech_key = config["AZURE_SPEECH_KEY"]
service_region = config["AZURE_SPEECH_REGION"]

# Create a speech configuration
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# Enable auto language identification and specify candidate languages
auto_detect_source_language_config = speechsdk.languageconfig.AutoDetectSourceLanguageConfig(
    languages=["zh-CN", "en-US", "de-DE"] # Replace with the languages you want to detect
)

# Create a recognizer with auto language identification
audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)  # Use default microphone
speech_recognizer = speechsdk.SpeechRecognizer(
    speech_config=speech_config,
    auto_detect_source_language_config=auto_detect_source_language_config,
    audio_config=audio_config,
)

print("Speak into your microphone to detect the language...")

# Start language identification and recognition
def recognized_callback(evt):
    result = evt.result
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        detected_language = result.properties[
            speechsdk.PropertyId.SpeechServiceConnection_AutoDetectSourceLanguageResult
        ]
        print(f"Detected Language: {detected_language}")
        print(f"Recognized Text: {result.text}")
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized.")

speech_recognizer.recognized.connect(recognized_callback)

# Start continuous recognition
speech_recognizer.start_continuous_recognition()
print("Listening... Press Ctrl+C to stop.")

try:
    while True:
        pass
except KeyboardInterrupt:
    print("Stopping...")
    speech_recognizer.stop_continuous_recognition()
